In [1]:
# PySpark sessions for CPU and GPU
from pyspark.sql import SparkSession

# for CPU
"""
spark = SparkSession.builder \
	.master("local").appName("hdfs_test")\
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9009") \
    .getOrCreate()
"""

# for GPU
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("hdfs_test") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.task.cpus", "4") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.rapids.memory.gpu.pool.size", "6G") \
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.maxResultSize", "16g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/06 16:16:29 WARN Utils: Your hostname, MyPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/06 16:16:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/06 16:16:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/06 16:16:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
silver_layer_path = "file:///home/tim/al"
reviews_embedded = spark.read.parquet(f"{silver_layer_path}/sampled_review_info_embedded")
print(reviews_embedded.count())
reviews_embedded.show(3)

25/05/06 16:16:32 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/05/06 16:16:32 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/05/06 16:16:32 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


1101910
+--------+--------------------+--------------------+------------+
| book_id|             user_id|text_embedded_vector|    faiss_id|
+--------+--------------------+--------------------+------------+
|23341863|81ec8d766fc3a5ebd...|[0.036919385, -0....|523986010112|
|17315867|81ec8d766fc3a5ebd...|[-0.07486253, 0.0...|523986010113|
|18165974|81ec8d766fc3a5ebd...|[-0.044150714, -0...|523986010114|
+--------+--------------------+--------------------+------------+
only showing top 3 rows



In [ ]:
import numpy as np
import faiss

EMD_COL_NAME = "text_embedded_vector"

first_vector = reviews_embedded.select(EMD_COL_NAME).first()[EMD_COL_NAME]
DIM = len(first_vector)
index = faiss.IndexFlatIP(DIM)


rdd = reviews_embedded.select(EMD_COL_NAME).rdd

batch = []
batch_size = 50000
count = 0

print("Starting streamed FAISS indexing...")

for row in rdd.toLocalIterator():
    vec = row[EMD_COL_NAME]
    batch.append(vec)
    count += 1

    if len(batch) == batch_size:
        np_batch = np.array(batch, dtype=np.float32)
        np_batch /= np.linalg.norm(np_batch, axis=1, keepdims=True)
        index.add(np_batch)
        print(f"🔹 Indexed {count} vectors")
        batch.clear()

# Final leftover batch
if batch:
    np_batch = np.array(batch, dtype=np.float32)
    np_batch /= np.linalg.norm(np_batch, axis=1, keepdims=True)
    index.add(np_batch)
    print(f"🔹 Indexed final {len(batch)} vectors")

# Save FAISS
faiss.write_index(index, "faiss_reviews_index.index")
print("✅ FAISS index saved.")



🚀 Starting streamed FAISS indexing...


25/05/06 16:16:39 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


🔹 Indexed 50000 vectors


25/05/06 16:16:44 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


🔹 Indexed 100000 vectors


...

🔹 Indexed 950000 vectors


25/05/06 16:17:36 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


🔹 Indexed 1000000 vectors
🔹 Indexed 1050000 vectors


25/05/06 16:17:40 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


🔹 Indexed 1100000 vectors
🔹 Indexed final 1910 vectors
✅ FAISS index saved.
